In [40]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F

In [44]:
import nltk 
from nltk.corpus import brown
#for sent in brown.sents():
#    print(" ".join(sent))

In [32]:
corpus = ['rock cab',
'ren cab',
'us cab',
'howagirepiguros.com wig',
'stochost street',
'skipholt street',
'11.02.2010.v.m ski',
'bansko ski',
'2008 paddle',
'gf paddle',
'olivier trolleybus']

In [33]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)
print tokenized_corpus

[['rock', 'cab'], ['ren', 'cab'], ['us', 'cab'], ['howagirepiguros.com', 'wig'], ['stochost', 'street'], ['skipholt', 'street'], ['11.02.2010.v.m', 'ski'], ['bansko', 'ski'], ['2008', 'paddle'], ['gf', 'paddle'], ['olivier', 'trolleybus']]


In [74]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)
print idx2word 

{0: 'rock', 1: 'cab', 2: 'ren', 3: 'us', 4: 'howagirepiguros.com', 5: 'wig', 6: 'stochost', 7: 'street', 8: 'skipholt', 9: '11.02.2010.v.m', 10: 'ski', 11: 'bansko', 12: '2008', 13: 'paddle', 14: 'gf', 15: 'olivier', 16: 'trolleybus'}


In [98]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array
#print idx_pairs

In [99]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [100]:
embedding_dims = 100
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 200
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data[0]
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print({loss_val/len(idx_pairs)})
        #print len(idx_pairs)

set([18.695258398060602])
set([15.113533755922452])
set([12.032077604045215])
set([9.530654434772838])
set([7.4193262688607895])
set([5.7225959475763375])
set([4.394397540369854])
set([3.3934644777208716])
set([2.5961881937797773])
set([1.9698766886444488])
set([1.495507491703558])
set([1.1377430367124775])
set([0.8979108029783344])
set([0.7488006034545833])
set([0.6421889426882527])
set([0.5522812234822513])
set([0.47971205399980216])
set([0.43656587660635443])
set([0.4171381910723126])
set([0.40719163159585814])


In [106]:
v = W1
u = W2
def cosine_distance(u, v):
    return numpy.dot(u, v) / (math.sqrt(numpy.dot(u, u)) * math.sqrt(numpy.dot(v, v)))

In [116]:
def cosdis(v, u):
    # which characters are common to the two words?
    common = v[1].intersection(u[1])
    
    # by definition of cosine distance we have
    return sum(v[0][ch]*u[0][ch] for ch in common)/v[2]/v[2]

In [121]:
 #print cosdis('1','2')
 #print cosine_distance('2323','3232')  

In [124]:
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

In [126]:
def cosdis(v, u):
    # which characters are common to the two words?
    common = v[1].intersection(u[1])
    
    # by definition of cosine distance we have
    return sum(v[0][ch]*u[0][ch] for ch in common)/v[2]/v[2]

In [133]:
#a = 'safasfeqefscwaeeafweeaeawaw'
#b = 'tsafdstrdfadsdfdswdfafdwaed'
#c = 'optykop;lvhopijresokpghwji7'
a ='cab'
b='street'

In [134]:
va = word2vec(a)
vb = word2vec(b)
vc = word2vec(c)

In [135]:
print cosdis(va,vb)

0.0
